In [7]:
import numpy as np
import pandas as pd
from pymysql import *
host_name = '10.2.1.170'
port_name = 5506
user_name = 'ai_account'
psw = 'qvufj2nj'

In [2]:
def table_from_mysql(database_name,table_name,where_str = None,select_column = None):
    conn=connect(host = host_name,port = port_name,user = user_name, password = psw,charset = 'utf8',database = database_name)
    cs1 = conn.cursor()
    
    sql = 'select * from %s' % table_name
    if isinstance(select_column,str) and len(select_column) > 0:
        sql = 'select %s from %s' % (select_column,table_name)
    if  isinstance(where_str,str):
        sql = sql  + where_str
    count = cs1.execute(sql)
    result = [list(column_set) for column_set in list(cs1.fetchall())]

    count2 = cs1.execute('SHOW FULL FIELDS FROM %s' % table_name)
    result2 = list(cs1.fetchall())
    columns = [column_set[0] for column_set in result2]
    if isinstance(select_column,str) and len(select_column) > 0:
        columns = select_column.split(',')
    cs1.close()
    conn.close()
    print("查询到%d条数据:" % count)
    return result,columns

In [9]:
import time
import datetime

from pandas.tseries.offsets import Day, MonthEnd

t = time.time()

print (t)                       #原始时间数据
print (int(t))                  #秒级时间戳
print (int(round(t * 1000)))    #毫秒级时间戳
nowTime = lambda:int(round(t * 1000))
print (nowTime());              #毫秒级时间戳，基于lambda

last_month = datetime.datetime.now()-MonthEnd(1)
last_month = last_month.strftime('%Y%m')#日期格式化'%Y-%m-%d %H:%M:%S'

1554892077.1980367
1554892077
1554892077198
1554892077198


# 基本属性 分子公司-工号-姓名等
- id	用户ID 
- login_name	用户登录名，也是用户的工号
- login_pwd	登录密码密文
- locked	帐号是否被锁定:0、正常；1、锁定；2、离职；默认：0、正常
- real_name	用户真实姓名
- phone	电话
- org_id	不可为空，无则为0
- sex	性别1男0女
- description	备注
- work_area	工作区域  9
- seat_number	外呼号
- email	邮箱地址
- entry_date	入职时间
- dimission_time	离职时间 13
- auth_id	鉴权中心ID
- work_age	工龄
- state_type	在线状态
- creater_id	创建人Id
- creater_name	创建人名称
- creater_org_id	
- create_time	创建时间
- updater_id	更新人Id
- post	岗位
- leader	上级
- rank	职级
- race	族类
- work_phone	工作手机
- hide_phone	外呼隐号
- voip_username	电信VOIP帐号
- show_tel	电信外呼号码
- voip_pwd	电信VOIP密码

In [10]:
data,columns = table_from_mysql('db_iboss2','sys_user_info',select_column=','.join(['login_name','org_id','sex','work_area','entry_date','work_age','id_card']))
sys_user_info = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
login_number = len(set(sys_user_info['login_name'].values.tolist()))
print('有 %d 个员工' % login_number)
sys_user_info.head()

查询到30766条数据:
有 30766 个员工


,login_name,org_id,sex,work_area,entry_date,work_age,id_card
0,jrcrm_admin,1,1,WORK_AREA_CD,2017-01-01 00:00:00,7.0,None
1,dropper,1,None,WORK_AREA_CD,2017-01-01 00:00:00,3.0,222222222222222222
2,order_admin,68,1,None,2017-01-01 00:00:00,5.0,None
3,hdc,9042,1,WORK_AREA_CD,2017-01-01 00:00:00,3.0,None
4,xmt_admin,83,1,WORK_AREA_CD,2017-01-01 00:00:00,3.0,None


In [11]:
# x = [0]+[i for i in range(2,6)] + [8,10,11,13,14] + [i for i in range(16,len(sys_user_info.columns))]
# sys_user_info.drop(sys_user_info.columns[x], axis=1, inplace=True)
print(sys_user_info.shape)
login_number = len(set(sys_user_info['login_name'].values.tolist()))
print('有 %d 个员工' % login_number)
sys_user_info.head()

(30766, 7)
有 30766 个员工


,login_name,org_id,sex,work_area,entry_date,work_age,id_card
0,jrcrm_admin,1,1,WORK_AREA_CD,2017-01-01 00:00:00,7.0,None
1,dropper,1,None,WORK_AREA_CD,2017-01-01 00:00:00,3.0,222222222222222222
2,order_admin,68,1,None,2017-01-01 00:00:00,5.0,None
3,hdc,9042,1,WORK_AREA_CD,2017-01-01 00:00:00,3.0,None
4,xmt_admin,83,1,WORK_AREA_CD,2017-01-01 00:00:00,3.0,None


# 员工部门信息
- CREATE TABLE `sys_org_main` (
-   `id` bigint(20) NOT NULL COMMENT '组织机构ID',
-   `code` varchar(300) DEFAULT NULL COMMENT '组织机构编码',
-   `name` varchar(200) DEFAULT NULL COMMENT '组织机构名称',
-   `description` varchar(600) DEFAULT NULL COMMENT '描述',
-   `sort` int(11) DEFAULT NULL COMMENT '排序',
-   `leader_id` bigint(20) DEFAULT NULL COMMENT '部门负责人ID',
-   `leader_name` varchar(255) DEFAULT NULL COMMENT '部门负责人名称',
-   `org_level` varchar(255) DEFAULT NULL COMMENT '层级(分公司，事业部，中心，商务部门，其他)',
-   `type` varchar(255) DEFAULT '' COMMENT '类型（加盟商、直营、联营）',
-   `bus_type` text COMMENT '绑定业态',
-   `is_receive` tinyint(255) DEFAULT '0' COMMENT '是否可接单 0不可接单，1可接单，2抢单',
-   `enabled` tinyint(4) DEFAULT NULL COMMENT '是否启用：1，启用；0，禁用',
-   `place_code` varchar(254) DEFAULT NULL COMMENT '所属地区code',
-   `create_time` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',
-   `creater_id` bigint(20) DEFAULT NULL COMMENT '创建人标识',
-   `creater_name` varchar(255) DEFAULT NULL COMMENT '创建人名称',
-   `update_time` timestamp NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '修改时间',
-   `updater_id` bigint(20) DEFAULT NULL COMMENT '修改人标识',
-   `updater_name` varchar(255) DEFAULT NULL COMMENT '修改人名称',
-   `DB_UPDATE_TIME` datetime DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
-   `upload_type` int(1) DEFAULT NULL,
-   `upload_time` bigint(20) DEFAULT NULL,
-   `upload_origin` varchar(50) DEFAULT NULL,
-   PRIMARY KEY (`id`) USING BTREE
- ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='组织机构';

In [12]:
data,columns = table_from_mysql('db_dwh','dw_dept')
dw_dept = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
dw_dept.tail()

查询到4102条数据:


,GROUP_ID,COMPANY_ID,COMPANY_NAME,BUSINESS_DEPT_ID,BUSINESS_DEPT_NAME,CENTER_ID,CENTER_NAME,DEPT_ID,DEPT_NAME,GROUP_NAME,LEVEL,FLAG,TYPE,UPDATE_TIME
4097,7801954858097762304,83,成都分公司,161,成都融资事业部,7746160817448738816,成都融资自创中心,7801954858097762304,成都融资自创七部,成都融资自创七部,4,0,0,2019-04-10 01:30:00
4098,7801955614850867200,83,成都分公司,161,成都融资事业部,7746160817448738816,成都融资自创中心,7801955614850867200,成都融资自创八部,成都融资自创八部,4,0,0,2019-04-10 01:30:00
4099,7801955615375155200,83,成都分公司,161,成都融资事业部,7746160817448738816,成都融资自创中心,7801955615375155200,成都融资自创九部,成都融资自创九部,4,0,0,2019-04-10 01:30:00
4100,7801955615685533696,83,成都分公司,161,成都融资事业部,7746160817448738816,成都融资自创中心,7801955615685533696,成都融资自创十部,成都融资自创十部,4,0,0,2019-04-10 01:30:00
4101,7803418882625822720,7692259833608802304,长沙分公司,7725636182275702784,长沙互联网营销事业部,7725636550740459520,长沙互联网营销商务中心,7803418882625822720,长沙互联网营销商务中心商务三部,长沙互联网营销商务中心商务三部,4,0,0,2019-04-10 01:30:00


In [13]:
data,columns = table_from_mysql('db_dwh','org_type')
org_type = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
org_type.tail()

查询到126条数据:


,company_id,company_name,business_dept_id,business_dept_name,type,bigyt,smallyt,smallyt_code,UPDATE_TIME
121,7686796881606873088,宜昌分公司,7741068646793134080,宜昌法律事业部,1,知产项目法律版块,法律,BUS_YT_FL,2019-03-01 11:09:37
122,7682365156810166272,佛山分公司,7746141433499795456,佛山会计事业部,1,创业板块,会计,BUS_YT_KJ,2019-03-01 11:07:47
123,385,北京分公司,7771229445311356928,北京海淀创业发展事业部,1,创业版块,工商,BUS_YT_CY,2019-03-01 11:19:15
124,385,北京分公司,7775877413297512448,北京海淀融资事业部,1,融资业态,融资,BUS_YT_DK,2019-03-01 11:19:52
125,9030,集团电销中心,7776262458531962880,集团电销资质事业部,1,资质业态,资质,BUS_YT_ZZ,2019-03-01 11:19:33


In [14]:
'''        select a.* c.smallyt 
        from db_iboss2.sys_user_info a left join db_dwh.dw_dept b on a.org_id=b.group_id left join db_dwh.org_type c on b.BUSINESS_DEPT_ID=c.business_dept_id 
        from db_iboss2.sys_user_info a,table2 db_dwh.dw_dept,db_dwh.org_type c where a.org_id=b.group_id and b.BUSINESS_DEPT_ID=c.business_dept_id '''
dw_dept['business_dept_id'] = dw_dept['BUSINESS_DEPT_ID']
business_dept = pd.merge(dw_dept, org_type, on='business_dept_id')
x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22]
business_dept.drop(business_dept.columns[x], axis=1, inplace=True)
print(business_dept.shape)
print(business_dept['smallyt'].value_counts())
business_dept = business_dept[business_dept['smallyt'] == '融资']
print('融资有 %d 个不同的GROUP_ID' % len(set(business_dept['GROUP_ID'].values.tolist())))
business_dept.head()

(2790, 2)
融资      651
工商      447
法律      350
资质      302
会计      292
知产      285
互联网     142
综合       90
财税       59
金融发展     51
培训       41
人事外包     37
品牌       22
项目       21
Name: smallyt, dtype: int64
融资有 651 个不同的GROUP_ID


,GROUP_ID,smallyt
776,161,融资
777,162,融资
778,197,融资
779,200,融资
780,203,融资


In [15]:
business_dept['org_id'] = business_dept['GROUP_ID']
sys_user_info = pd.merge(sys_user_info, business_dept, on='org_id')
sys_user_info.drop(index=0,axis=0,inplace=True)# 第一条是合规专员
print(sys_user_info['work_area'].value_counts())
sys_user_info.head()

WORK_AREA_CD     3475
WORK_AREA_BJ      569
WORK_AREA_CQ      540
WORK_AREA_GZ      376
WORK_AREA_WH      322
WORK_AREA_HZ      279
WORK_AREA_SZ      212
WORK_AREA_CS      193
WORK_AREA_FS      136
WORK_AREA_ZZ      125
WORK_AREA_SH       95
WORK_AREA_SJZ      94
WORK_AREA_DG       53
WORK_AREA_YC       34
12                  1
Name: work_area, dtype: int64


,login_name,org_id,sex,work_area,entry_date,work_age,id_card,GROUP_ID,smallyt
1,2101781,197,0,WORK_AREA_CD,2017-01-01 00:00:00,3.0,None,197,融资
2,2101922,197,1,WORK_AREA_CD,2018-09-06 00:00:00,6.0,511523198711153556,197,融资
3,2101935,197,0,WORK_AREA_CD,2017-01-01 00:00:00,3.0,510902198508239505,197,融资
4,2102182,197,1,WORK_AREA_CD,2017-01-01 00:00:00,3.0,433130199006180418,197,融资
5,2102379,197,0,WORK_AREA_CD,2016-01-04 00:00:00,3.0,511321199308093734,197,融资


In [16]:
x = [1,4,6,7]
sys_user_info.drop(sys_user_info.columns[x], axis=1, inplace=True)
print(sys_user_info.shape)
sys_user_info.head()

(6588, 5)


,login_name,sex,work_area,work_age,smallyt
1,2101781,0,WORK_AREA_CD,3.0,融资
2,2101922,1,WORK_AREA_CD,6.0,融资
3,2101935,0,WORK_AREA_CD,3.0,融资
4,2102182,1,WORK_AREA_CD,3.0,融资
5,2102379,0,WORK_AREA_CD,3.0,融资


In [17]:
for i in tqdm(range(len(sys_user_info))):
    print(sys_user_info.iloc[i]['login_name'])
#     df.at[i,'login_name'] = re.search(r"\d+", str(df.iloc[i]['login_name'])).group() 

NameError: name 'tqdm' is not defined

# 每月日均上门量
- id
- sys_type 系统来源'IBOSS2'与''小顶云  iboss2 代表外出，cloud代表上门
- dw_month	小顶云为预约月份，IBOSS2预约与到访为同一时间
- dw_date 预约日期
- promissory_time 预约时间
- visit_month	到访月
- visit_date 到访日期
- visit_time 到访时间
- send_note_time 发送短信时间
- user_name	用户名
- user_no 用户工号
- customer_name	小顶云为客户名，iboss2 为客户编号
- cause	到访类型，都为业务办理
- address 到访地址
- status '状态，0：待到访，1：已到访，2：取消'
- status_name 状态名
- type_code 小顶云为业态名IBOSS2 为业态编码
- business_no 商机编号
- DB_UPDATE_TIME

In [18]:
data,columns = table_from_mysql('bi_behavior','bi_visit_detail')
bi_visit_detail = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
bi_visit_detail = bi_visit_detail[(bi_visit_detail['status_name'] == '已到访') |(bi_visit_detail['type_code'] == '信用贷款') | (bi_visit_detail['type_code'] == '信贷小贷') | (bi_visit_detail['type_code'] ==  '信贷小贷（禁）')]
print(bi_visit_detail['type_code'].value_counts())
print('过滤后的数据：',bi_visit_detail.shape)
bi_visit_detail.head()

查询到88052条数据:
信用贷款                          20298
BUS_YT_DK_XYDK                 5707
                               3028
信贷小贷                           2822
民事诉讼                           1843
注册服务                           1442
BUS_YT_FL_MSSS                 1140
BUS_YT_CY_ZCFW                  970
非诉                              925
BUS_YT_FL_FS                    720
BUS_YT_DK_XYDK_1                675
信贷小贷（禁）                         584
资质/施工资质                         376
商标基础                            375
BUS_YT_ZSCQ_SB                  306
注销业务                            294
证书/证书挂靠                         278
变更服务                            242
资质/其他资质                         213
微信业务                            212
会计代理                            211
BUS_YT_ZZ_SGZZ                  207
互联网资质                           203
专利基础                            170
网站建设                            166
社保公积金代理                         166
BUS_YT_HLW_JJCP                 155
BUS_YT_CY_BGFW 

,id,sys_type,dw_month,dw_date,promissory_time,visit_month,visit_date,visit_time,send_note_time,user_name,...,customer_id,customer_no,customer_name,cause,address,status,status_name,type_code,business_no,DB_UPDATE_TIME
1,00023333-41ae-4dbf-b595-0292f8764226,cloud,201812.0,20181212.0,2018-12-12 19:16:23,NaN,NaN,NaT,2018-12-12 18:16:29,陈秘,...,NaN,None,张正伟,融资业务,渝北区红锦大道59号嘉州协信中心A栋20-1,0,待到访,信用贷款,SJ201812044213,2019-01-15 20:08:54
2,00039388-c0c2-47d5-8fe6-15b634ce9bce,cloud,201901.0,20190123.0,2019-01-23 14:30:00,NaN,NaN,NaT,2019-01-22 15:12:20,赖维圣,...,NaN,None,曹红,融资业务,天河区天河东路67号丰兴广场A栋21楼,0,待到访,信用贷款,SJ201901180153,2019-01-22 23:30:00
4,000a7f0d-a817-4645-962a-268b0b76d41e,cloud,201812.0,20181225.0,2018-12-25 10:45:43,NaN,NaN,NaT,2018-12-24 10:46:00,郑先海,...,NaN,None,向老师,融资业务,渝北区红锦大道59号嘉州协信中心A栋20-1,0,待到访,信用贷款,SJ201812240227,2019-01-15 20:08:54
5,000db180-f2b7-4bf4-89f5-707c34b7a256,cloud,201902.0,20190226.0,2019-02-26 14:00:00,201902.0,20190226.0,2019-02-26 14:20:39,2019-02-26 10:58:09,彭仕琼,...,NaN,None,白显林,业务办理,渝北区红锦大道59号嘉州协信中心A栋20-1,1,已到访,信用贷款,ZY2019022505185,2019-02-26 23:30:00
6,0010af40-abcf-4d71-b1e4-c1539284d02e,cloud,201812.0,20181227.0,2018-12-27 11:00:20,NaN,NaN,NaT,2018-12-24 11:00:37,雷诚鑫,...,NaN,None,王兴鹏,融资业务,渝北区红锦大道59号嘉州协信中心A栋20-1,0,待到访,信用贷款,SJ201812241098,2019-01-15 20:08:54


In [19]:
bi_visit_detail['login_name'] = bi_visit_detail['user_no']
bi_visit_detail = pd.merge(bi_visit_detail, sys_user_info, on='login_name', how = 'right')
print(bi_visit_detail.shape)
bi_visit_detail.head()

(40253, 26)


,id,sys_type,dw_month,dw_date,promissory_time,visit_month,visit_date,visit_time,send_note_time,user_name,...,status,status_name,type_code,business_no,DB_UPDATE_TIME,login_name,sex,work_area,work_age,smallyt
0,00023333-41ae-4dbf-b595-0292f8764226,cloud,201812.0,20181212.0,2018-12-12 19:16:23,NaN,NaN,NaT,2018-12-12 18:16:29,陈秘,...,0.0,待到访,信用贷款,SJ201812044213,2019-01-15 20:08:54,952859647,0,WORK_AREA_CQ,8.0,融资
1,100e53d7-cf24-433a-b01e-25bb23397a8c,cloud,201901.0,20190116.0,2019-01-16 14:00:00,201901.0,20190116.0,2019-01-16 14:39:54,2019-01-16 11:20:45,陈秘,...,1.0,已到访,信贷小贷,SJ201808097555,2019-01-16 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资
2,12e68d44-8992-4ca6-b8ab-4a97ec6b651e,cloud,201901.0,20190107.0,2019-01-07 14:30:00,201901.0,20190107.0,2019-01-07 14:00:53,2019-01-07 11:16:29,陈秘,...,1.0,已到访,信用贷款,SJ201811181510,2019-01-15 20:08:54,952859647,0,WORK_AREA_CQ,8.0,融资
3,150fb5ac-d280-4456-8a9d-938b77cb2896,cloud,201901.0,20190125.0,2019-01-25 09:00:00,NaN,NaN,NaT,2019-01-24 14:31:30,陈秘,...,2.0,取消,信用贷款,SJ201901231968,2019-01-25 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资
4,1a3f89d8-aa01-4bc6-9e3a-54acb8392b53,cloud,201901.0,20190128.0,2019-01-28 10:30:00,NaN,NaN,NaT,2019-01-28 09:44:44,陈秘,...,0.0,待到访,信用贷款,SJ201810121580,2019-01-28 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资


In [20]:
x = [0,1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18]
bi_visit_detail.drop(bi_visit_detail.columns[x], axis=1, inplace=True)
print(bi_visit_detail.shape)
bi_visit_detail['visit_count'] = 1
bi_visit_detail.fillna(method='bfill',inplace=True)
bi_visit_detail.head()

(40253, 8)


,visit_month,business_no,DB_UPDATE_TIME,login_name,sex,work_area,work_age,smallyt,visit_count
0,201901.0,SJ201812044213,2019-01-15 20:08:54,952859647,0,WORK_AREA_CQ,8.0,融资,1
1,201901.0,SJ201808097555,2019-01-16 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资,1
2,201901.0,SJ201811181510,2019-01-15 20:08:54,952859647,0,WORK_AREA_CQ,8.0,融资,1
3,201902.0,SJ201901231968,2019-01-25 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资,1
4,201902.0,SJ201810121580,2019-01-28 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资,1


In [21]:
bi_visit_detail = bi_visit_detail[bi_visit_detail['visit_month'] == int(last_month)]
print(bi_visit_detail.shape)
bi_visit_detail.head()

(11580, 9)


,visit_month,business_no,DB_UPDATE_TIME,login_name,sex,work_area,work_age,smallyt,visit_count
35,201903.0,ZY2019022809745,2019-03-01 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资,1
40,201903.0,SJ201901045365,2019-01-15 23:30:00,952859647,0,WORK_AREA_CQ,8.0,融资,1
41,201903.0,ZY2019030613879,2019-04-01 15:55:21,952859647,0,WORK_AREA_CQ,8.0,融资,1
42,201903.0,ZY2019030612235,2019-04-01 15:55:21,952859647,0,WORK_AREA_CQ,8.0,融资,1
43,201903.0,ZY2019031101935,2019-04-01 15:55:21,952859647,0,WORK_AREA_CQ,8.0,融资,1


In [22]:
#bi_visit_detail.groupby(['login_name','business_no','visit_month']).agg({'count':[np.size,np.sum]})#去重
# bi_visit_detail['count'] = 1
last_month_df = bi_visit_detail.groupby(['login_name']).agg({'visit_count':np.sum}).reset_index()
print(last_month_df.shape)
last_month_df.drop_duplicates(inplace=True)
print(last_month_df.shape)
last_month_df.head()

(817, 2)
(817, 2)


,login_name,visit_count
0,100008,46
1,100017,44
2,100045,8
3,100066,11
4,100091,20


In [23]:
df = pd.merge(sys_user_info, last_month_df, on='login_name', how = 'left')
print(df.shape)
df.drop_duplicates(inplace=True)
print('还剩 %d 个员工有记录' % len(df['login_name'].value_counts().tolist()))
df.head()

(6588, 6)
还剩 6588 个员工有记录


,login_name,sex,work_area,work_age,smallyt,visit_count
0,2101781,0,WORK_AREA_CD,3.0,融资,NaN
1,2101922,1,WORK_AREA_CD,6.0,融资,NaN
2,2101935,0,WORK_AREA_CD,3.0,融资,NaN
3,2102182,1,WORK_AREA_CD,3.0,融资,NaN
4,2102379,0,WORK_AREA_CD,3.0,融资,NaN


# 每月日均电话量
- DW_YEAR	年
- DW_MONTH	月
- DW_DATE	日期(数字格式例如:20170101)
- USER_I	商务ID
- USER_NO	
- USER_NAME	姓名
- SUM_SECOND	呼叫总时长(秒)
- SUM_CALL	呼出总数
- SUCCESS_CALL	当日呼出接通总数
- 30S_CALL	30秒以上呼出有效通话
- 60S_CALL	60秒以上呼出有效通话
- SUM_CALLED	呼入总数
- SUCCESS_CALLED	当日呼入接通总数
- 30S_CALLED	30秒以上呼入有效通话
- 60S_CALLED	60秒以上呼入有效通话
- UPDATE_TIME

In [24]:
data,columns = table_from_mysql('db_call','dw_d_called')
dw_d_called = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
dw_d_called['login_name'] = dw_d_called['USER_NO']
dw_d_called = dw_d_called[dw_d_called['DW_MONTH'] == int(last_month)]
#dw_d_called = dw_d_called['']
print(dw_d_called.shape)
dw_d_called.head()

查询到1337394条数据:
(62616, 17)


,DW_YEAR,DW_MONTH,DW_DATE,USER_ID,USER_NO,USER_NAME,SUM_SECOND,SUM_CALL,SUCCESS_CALL,30S_CALL,60S_CALL,SUM_CALLED,SUCCESS_CALLED,30S_CALLED,60S_CALLED,UPDATE_TIME,login_name
1255886,2019,201903,20190301,998.0,2180105,蒋春方,1975.0,16,14,14,12,0,0,0,0,2019-04-02 08:59:25,2180105
1255887,2019,201903,20190301,1006.0,2180178,但韬,189.0,21,2,1,1,0,0,0,0,2019-04-02 08:59:25,2180178
1255888,2019,201903,20190301,1009.0,2180224,程翔,704.0,1,1,1,1,0,0,0,0,2019-04-02 08:59:25,2180224
1255889,2019,201903,20190301,1015.0,2180287,曾海,108.0,2,2,1,1,0,0,0,0,2019-04-02 08:59:25,2180287
1255890,2019,201903,20190301,1036.0,2180569,赵莲,362.0,6,6,3,3,0,0,0,0,2019-04-02 08:59:25,2180569


In [25]:
x = [0,1,2,3,4,5,6,7,9,10,11,12,13,14,15]
dw_d_called.drop(dw_d_called.columns[x], axis=1, inplace=True)
print(dw_d_called.shape)
dw_d_called.head()

(62616, 2)


,SUCCESS_CALL,login_name
1255886,14,2180105
1255887,2,2180178
1255888,1,2180224
1255889,2,2180287
1255890,6,2180569


In [26]:
last_month_df = dw_d_called.groupby(['login_name']).agg({'SUCCESS_CALL':np.sum})
dw_d_called = pd.merge(dw_d_called, last_month_df, on='login_name', how = 'left')
print(dw_d_called.shape)
last_month_df.head()

(62616, 3)


,SUCCESS_CALL
login_name,
100008,730
100012,13
100014,512
100017,1058
100027,279


In [27]:
print(dw_d_called.head())
dw_d_called.drop(dw_d_called.columns[[0]], axis=1, inplace=True)
dw_d_called.head()

   SUCCESS_CALL_x login_name  SUCCESS_CALL_y
0              14    2180105             416
1               2    2180178             149
2               1    2180224              63
3               2    2180287              67
4               6    2180569              19


,login_name,SUCCESS_CALL_y
0,2180105,416
1,2180178,149
2,2180224,63
3,2180287,67
4,2180569,19


In [28]:
df = pd.merge(dw_d_called, df, on='login_name', how = 'right')
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(22930, 7)
(6588, 7)


,login_name,SUCCESS_CALL_y,sex,work_area,work_age,smallyt,visit_count
0,2100370,129.0,1,WORK_AREA_CD,61.0,融资,NaN
20,2102642,307.0,0,WORK_AREA_CD,36.0,融资,NaN
32,2100932,11.0,1,WORK_AREA_CD,52.0,融资,NaN
37,3061288,220.0,0,WORK_AREA_CD,48.0,融资,2.0
60,2101439,924.0,0,WORK_AREA_CD,36.0,融资,8.0


# 工时
- kq_date	日期
- dept_oneid	
- dept_onename
- dept_twoid
- dept_twoname
- dept_threeid	
- dept_threename	
- dept_id	
- deptname	
- user_name	工号
- true_name	商务
- job_name	岗位
- first_dk	第一次打卡时间
- last_dk	最后一次打卡时间
- wqd	未签到
- wqt	未签退
- late	迟到
- working_hours	时长
- number_day	天数

In [29]:
data,columns = table_from_mysql('db_kq','d_report_kq',select_column=','.join(['kq_date','user_name','working_hours']))
d_report_kq = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
d_report_kq['login_name'] = d_report_kq['user_name']
d_report_kq['working_hours'].fillna(value = 7.5,inplace = True)
print(d_report_kq.shape)
d_report_kq.head()

查询到2278434条数据:
(2278434, 4)


,kq_date,user_name,working_hours,login_name
0,20171202,10045523,7.5,10045523
1,20171202,10054742,7.5,10054742
2,20171202,10054771,6.54,10054771
3,20171202,10055881,9.53,10055881
4,20171202,10064888,7.5,10064888


In [30]:
from tqdm import tqdm_notebook as tqdm
d_report_kq['kq_month'] = None
drop_index = []
for i in tqdm(range(len(d_report_kq))):
    month = str(d_report_kq.loc[i]['kq_date'])[:6]
    if str(last_month) == month:
        continue
    else:
        drop_index.append(i)
print(drop_index[:10])
d_report_kq.head()


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


,kq_date,user_name,working_hours,login_name,kq_month
0,20171202,10045523,7.5,10045523,None
1,20171202,10054742,7.5,10054742,None
2,20171202,10054771,6.54,10054771,None
3,20171202,10055881,9.53,10055881,None
4,20171202,10064888,7.5,10064888,None


In [31]:
d_report_kq.drop(axis=0,index = drop_index,inplace=True)
print(d_report_kq.shape)
d_report_kq['kq_month'] = last_month
d_report_kq.head()

(119802, 5)


,kq_date,user_name,working_hours,login_name,kq_month
2119362,20190301,908418675,8.06,908418675,201903
2119363,20190301,93879248,7.89,93879248,201903
2119364,20190301,944982315,8.81,944982315,201903
2119365,20190301,52398540,6.43,52398540,201903
2119366,20190301,30464538,10.83,30464538,201903


In [32]:
last_month_df = d_report_kq.groupby(['login_name']).agg({'working_hours':np.sum}).reset_index()
print(last_month_df.shape)
last_month_df.drop_duplicates(inplace=True)
print(last_month_df.shape)
last_month_df.head()

(6793, 2)
(6793, 2)


,login_name,working_hours
0,100004,0.00
1,100008,222.18
2,100010,151.22
3,100012,100.72
4,100014,208.50


In [33]:
# df['login_name'].apply(int) 
# df.applymap(str)


# bi_order_detail['login_name'].apply(str) #Convert a series
# df.applymap(str) #Convert the whole frame
# df = df.convert_objects(convert_numeric=True) 后会将object类型置为NaN
df = pd.merge(last_month_df, df, on='login_name', how = 'right')
print(df.shape)
df.head()

(6588, 8)


,login_name,working_hours,SUCCESS_CALL_y,sex,work_area,work_age,smallyt,visit_count
0,100008,222.18,730.0,0,WORK_AREA_WH,4.0,融资,46.0
1,100012,100.72,13.0,1,WORK_AREA_BJ,4.0,融资,NaN
2,100017,232.86,1058.0,1,WORK_AREA_CD,4.0,融资,44.0
3,100045,114.20,NaN,0,WORK_AREA_CQ,1.0,融资,8.0
4,100066,213.84,587.0,1,WORK_AREA_CD,4.0,融资,11.0


# 平均签单周期、每月签单数量
- id	子订单id
- sys_type	来源系统
- dw_month
- dw_date
- place_order_time	下单时间
- order_id	主订单id
- order_not	主订单编号
- order_business_no	子订单编号
- business_id	商机ID
- business_no	商机编号
- customer_id	客户id
- customer_no	客户编号
- customer_name	客户名称
- business_type_id	业态id
- business_type_name	业态名
- business_user_id	签单商务id
- user_no	
- business_user_name	签单商务名
- organization_id	商务部门id
- organization_name	商务部门名
- contract_no	合同号码
- sign_org_id	签单事业部id
- sign_org_name	签单事业部名
- status	订单状态
- bt_name	业态名
- bt_identifying	业态编码
- product_id	产品id
- serial_num	产品编号
- product_name	产品名
- product_amount	产品价格或者贷款金额
- affiliation_syb	归属事业部
- small_business	小业态id
- small_business_name	小业态名
- child_belong	生产归属事业部id
- child_belong_name	生产归属事业部

In [34]:
data,columns = table_from_mysql('bi_behavior','bi_order_detail',select_column=','.join(['dw_month','user_no','place_order_time']))
bi_order_detail = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
bi_order_detail['login_name'] = bi_order_detail['user_no']
bi_order_detail = bi_order_detail[bi_order_detail['dw_month'] == int(last_month)]
print(bi_order_detail.shape)
bi_order_detail.head()

查询到280153条数据:
(27544, 4)


,dw_month,user_no,place_order_time,login_name
252609,201903,66001792,2019-03-01 08:35:21,66001792
252610,201903,56127068,2019-03-01 08:47:11,56127068
252611,201903,56127068,2019-03-01 08:47:11,56127068
252612,201903,56127068,2019-03-01 08:50:32,56127068
252613,201903,56127068,2019-03-01 08:50:32,56127068


In [35]:
x = [0,1,2]
bi_order_detail.drop(bi_order_detail.columns[x], axis=1, inplace=True)
print(bi_order_detail.shape)
bi_order_detail.head()

(27544, 1)


,login_name
252609,66001792
252610,56127068
252611,56127068
252612,56127068
252613,56127068


In [36]:
bi_order_detail['每月签单次数'] = 1
# place_order_time比contract_no更有用，还能计算签单周期，contract_no有异常值
last_month_df = bi_order_detail.groupby(['login_name']).agg({'每月签单次数':np.sum}).reset_index()
print(last_month_df.shape)
last_month_df.drop_duplicates(inplace=True)
print(last_month_df.shape)
print(last_month_df['每月签单次数'].value_counts()[:5])
last_month_df.head()

(2428, 2)
(2428, 2)
1    341
2    234
3    194
4    167
5    155
Name: 每月签单次数, dtype: int64


,login_name,每月签单次数
0,100008,8
1,100014,7
2,100017,14
3,100027,3
4,100028,2


In [37]:
df = pd.merge(last_month_df, df, on='login_name', how = 'right')
print(df.shape)
df.head()

(6588, 9)


,login_name,每月签单次数,working_hours,SUCCESS_CALL_y,sex,work_area,work_age,smallyt,visit_count
0,100008,8.0,222.18,730.0,0,WORK_AREA_WH,4.0,融资,46.0
1,100017,14.0,232.86,1058.0,1,WORK_AREA_CD,4.0,融资,44.0
2,100045,6.0,114.20,NaN,0,WORK_AREA_CQ,1.0,融资,8.0
3,100066,3.0,213.84,587.0,1,WORK_AREA_CD,4.0,融资,11.0
4,100091,6.0,206.19,1119.0,1,WORK_AREA_BJ,4.0,融资,20.0


# 平均备注字数
- id	
- create_time	创建时间
- creater_id	创建人id
- creater_org_id	创建人部门id
- creater_name	姓名+工号
- update_time	最后修改时间
- updater_id	最后修改人id
- updater_org_id	最后修改人部门id
- updater_name	姓名+工号
- source_flag	外部系统来源标志,预留扩展字段，每外部系统占用一位，如1001
- read_flag 	外部系统读取标志,预留扩展字段，每外部系统占用一位，如1001
- customer_id	客户id
- table_name	业务表名
- table_id	业务id
- table_no	业务编号
- content	备注内容
- remark_type	备注类型，字典表
- remark_node	备注节点，字典表
- alt_json	json字符，不参与搜索
- from_user_id	商机来源人id
- from_user_org_id	商机来源人组织id
- to_user_id	bigint(20)	商机去向人id
- to_user_org_id	商机去向人组织id

In [ ]:
data,columns = table_from_mysql('db_iboss2','uc_customer_record', 'where ',select_column=','.join(['update_time','creater_name','content']))
uc_customer_record = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
#uc_customer_record = uc_customer_record['']
uc_customer_record.head()

#  业绩 签单业绩 协单业绩 转介绍业绩 业绩合计
- performance_month	业绩月
- performance_date	业绩日
- sys_type	系统来源
- id	对应系统的业绩表ID
- create_time	创建时间
- performance_time	业绩时间
- place_time	下单时间
- sign_user_id	商务id
- sign_user_name	商务名
- user_no	工号
- sign_org_id	商务部门id
- sign_org_name	商务部门名
- performance_amount	业绩
- profit_amount	利润
- internal_cost	内部成本
- external_cost	外部成本
- payment_amount	核款金额
- business_amount	订单金额
- user_type	用户签单类型(签单或协单等)
- performence_type	业绩类型
- order_id	订单id
- order_no	订单编码
- customer_id	客户id
- customer_no	客户编码
- customer_name	客户名
- business_no	商机编码
- payment_id	核款id
- payment_no	客户编号
- type	核退款 1核款 2退款
- contract_no	合同号
- receipt_no	收据号
- order_type	订单类型
- payment_type	付款方式
- nature	type为1 核款时 值为 付款类型(首款等) 
- type为2 退款时 值为 退款退单类型 1 仅退款 2，内部结转,3退单退款
- add_type_code	资源新建类型
- opportunitytype_code	资源类型
- origin_code	资源渠道
- origin_type	资源几类
- is_cost	是否计算成本
- cost_price	成本价
- distribution_time	资源分配时间
- product_name	产品名称
- product_id	产品id
- serial_num	产品编号
- product_type	产品小业态
- product_type_parent	产品大业态

In [38]:
data,columns = table_from_mysql('bi_performance','bi_performance_detail',select_column=','.join(['performance_month','user_no','performance_amount']))
bi_performance_detail = pd.DataFrame(data, index=list(range(len(data))), columns=columns)
bi_performance_detail['login_name'] = bi_performance_detail['user_no']
bi_performance_detail = bi_performance_detail[bi_performance_detail['performance_month'] == int(last_month)]
print(bi_performance_detail.shape)
bi_performance_detail.head()

查询到230996条数据:
(39497, 4)


,performance_month,user_no,performance_amount,login_name
121512,201903,69278249,296.0000,69278249
122773,201903,97568436,7500.0000,97568436
128761,201903,40279083,1625.0000,40279083
128762,201903,10075136,1625.0000,10075136
128763,201903,40279083,50.0000,40279083


In [39]:
last_month_df = bi_performance_detail.groupby(['login_name']).agg({'performance_amount':np.sum}).reset_index()
print(last_month_df.shape)
last_month_df.drop_duplicates(inplace=True)
print(last_month_df.shape)
last_month_df.head()

(3219, 2)
(3219, 2)


,login_name,performance_amount
0,100008,23320.7300
1,100012,1925.0000
2,100014,4265.0000
3,100017,5367.9000
4,100027,6500.0000


In [40]:
df = pd.merge(last_month_df, df, on='login_name', how = 'right')
print(df.shape)
df.head()

(6588, 10)


,login_name,performance_amount,每月签单次数,working_hours,SUCCESS_CALL_y,sex,work_area,work_age,smallyt,visit_count
0,100008,23320.7300,8.0,222.18,730.0,0,WORK_AREA_WH,4.0,融资,46.0
1,100012,1925.0000,NaN,100.72,13.0,1,WORK_AREA_BJ,4.0,融资,NaN
2,100017,5367.9000,14.0,232.86,1058.0,1,WORK_AREA_CD,4.0,融资,44.0
3,100045,490.0000,6.0,114.20,NaN,0,WORK_AREA_CQ,1.0,融资,8.0
4,100066,1585.0000,3.0,213.84,587.0,1,WORK_AREA_CD,4.0,融资,11.0


In [41]:
import re
df['平均签单周期'] = 0
df['平均备注字数'] = 0
df['违规记录'] = 0
df['操行分'] = 0
df['每月日均意向客户数'] = 0
df['婚姻情况'] = 0
df['有无子女'] = 0
df['有无房贷'] = 0
df['有无车贷'] = 0
df['近三个月平均收入'] = 0
df['芝麻信用分'] = 0
df['有无子女'] = 0
df['商务级别'] = 0
df['日均新增微信好友数'] = 0
df['签单率'] = 0
df['放款率'] = 0
df['资源上门率'] = 0
df['客户属性填写'] = 0
df['是否离职'] = 0
for i in tqdm(range(len(df))):
    df.at[i,'login_name'] = re.search(r"\d+", str(df.iloc[i]['login_name'])).group() # 
    df.at[i,'平均签单周期'] = np.random.randint(low=0,high=20,size=1)[0] # 
    df.at[i,'平均备注字数'] = np.random.randint(low=0,high=10,size=1)[0] # 平均备注字数
    df.at[i,'违规记录'] = np.random.randint(low=0,high=2,size=1)[0] # 违规记录 0：没有记录 1：黄线 2：红线
    df.at[i,'操行分'] = np.random.randint(low=60,high=100,size=1)[0] # 操行分 每半年清零一次，每个月份的操行分不具有可比性
    df.at[i,'每月日均意向客户数'] = np.random.randint(low=0,high=10,size=1)[0] # 每月日均意向客户数
    df.at[i,'婚姻情况'] = np.random.randint(low=0,high=2,size=1)[0] # 婚姻情况 0：未知 1：未婚 2：已婚
    df.at[i,'有无子女'] = np.random.randint(low=0,high=2,size=1)[0] # 有无子女 0：未知 1：没有 2：有
    df.at[i,'有无房贷'] = np.random.randint(low=0,high=2,size=1)[0] # 房产情况 0：未知 1：没有 2：有
    df.at[i,'有无车贷'] = np.random.randint(low=0,high=2,size=1)[0] # 车 0：未知 1：没有 2：有
    df.at[i,'近三个月平均收入'] = np.random.randint(low=3000,high=30000,size=1)[0] # 负债 0：未知 1：没有 2：有
    df.at[i,'芝麻信用分'] = np.random.randint(low=0,high=750,size=1)[0] # 芝麻信用分 0：未知 
    df.at[i,'有无子女'] = np.random.randint(low=0,high=2,size=1)[0] # 有无子女 0：未知 1：有1个 2：大于1个
    df.at[i,'商务级别'] = np.random.randint(low=1,high=6,size=1)[0] # 私人手机使用 0：未知 1：没有 2：有
    df.at[i,'日均新增微信好友数'] = np.random.randint(low=0,high=10,size=1)[0] # 平均备注字数
    df.at[i,'签单率'] = np.random.random(3)[0] # 平均备注字数
    df.at[i,'放款率'] = np.random.random(3)[0] # 平均备注字数
    df.at[i,'资源上门率'] = np.random.random(3)[0] # 平均备注字数
    df.at[i,'客户属性填写'] = np.random.randint(low=0,high=5,size=1)[0] # 房产情况 0：未知 1：没有 2：有
    df.at[i,'是否离职'] = np.random.randint(low=0,high=1,size=1)[0] # 车 0：未知 1：没有 2：有
    
    
df.head()

,login_name,performance_amount,每月签单次数,working_hours,SUCCESS_CALL_y,sex,work_area,work_age,smallyt,visit_count,...,有无车贷,近三个月平均收入,芝麻信用分,商务级别,日均新增微信好友数,签单率,放款率,资源上门率,客户属性填写,是否离职
0,100008,23320.7300,8.0,222.18,730.0,0,WORK_AREA_WH,4.0,融资,46.0,...,1,14039,599,5,5,0,0,0,0,0
1,100012,1925.0000,NaN,100.72,13.0,1,WORK_AREA_BJ,4.0,融资,NaN,...,0,22371,217,4,3,0,0,0,0,0
2,100017,5367.9000,14.0,232.86,1058.0,1,WORK_AREA_CD,4.0,融资,44.0,...,1,23076,490,1,1,0,0,0,4,0
3,100045,490.0000,6.0,114.20,NaN,0,WORK_AREA_CQ,1.0,融资,8.0,...,1,5940,191,1,9,0,0,0,4,0
4,100066,1585.0000,3.0,213.84,587.0,1,WORK_AREA_CD,4.0,融资,11.0,...,0,15125,510,4,0,0,0,0,1,0


'30210455'

In [ ]:
df['login_name'].apply(int) 
df.applymap(str)
df.head()

In [42]:
writer = pd.ExcelWriter('商务模拟数据.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()